In [1]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [2]:
# If working in Colab, mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Direct to your project folder.
%cd /content/drive/Othercomputers/Computer1/dl-robocop

Mounted at /content/drive
/content/drive/Othercomputers/Computer1/dl-robocop


In [3]:
# Installing requirements
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 

In [4]:
from datasets import load_dataset
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import evaluate
import gensim
import transformers
import nltk
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
#import jsonlines

In [5]:
data_path = 'data/'
dataset = pd.read_csv(data_path + 'paradetox.tsv', sep='\t')

In [6]:
test_samples = dataset.iloc[1200:1210]
dataset = dataset.head(1000)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Define the maximum length and other parameters
max_length = 36

# Generate the input_ids and input_mask for each sample
input_ids = [torch.tensor(tokenizer.encode(d, max_length=max_length, padding='max_length')) for d in dataset['toxic']]
detoxified_ids = [torch.tensor(tokenizer.encode(d, max_length=max_length, padding='max_length')) for d in dataset['neutral1']]
input_mask = [torch.tensor([1 if token != tokenizer.pad_token_id else 0 for token in tokens]) for tokens in input_ids]

# Convert lists to tensors
input_ids_tensor = torch.stack(input_ids)
detoxified_ids_tensor = torch.stack(detoxified_ids)
input_mask_tensor = torch.stack(input_mask)

# Create a TensorDataset
tensor_dataset = TensorDataset(input_ids_tensor, detoxified_ids_tensor, input_mask_tensor)

NameError: name 'tokenizer' is not defined

In [ ]:
# PARADETOX MODEL
from transformers import BartForConditionalGeneration, AutoTokenizer
base_model_name = 'facebook/bart-base'
model_name = 'SkolkovoInstitute/bart-base-detox'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
# GPT2 MODEL
model_name = "vicgalle/gpt2-alpaca-gpt4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(model_name)
MAX_SEQ_LENGTH = tokenizer.model_max_length

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
# GREENLLAMA MODEL
# Note it is way too large for the RAM
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "UBC-NLP/GreenLLaMA-7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "

for (idx,input) in enumerate(dataset['toxic']):
  #input = prompt+input+"\n"

  input_ids = tokenizer(input, return_tensors="pt")
  label = dataset['neutral1'][idx]
  model = model.to(device)

  outputs = model.generate(**input_ids, max_new_tokens = 50, do_sample=False)
  print(f'Toxic sentence: {input}')
  print(f'Detoxified sentence: {tokenizer.decode(outputs[0], skip_special_tokens = True)}')
  print(f'Gold label: {label}\n')
  if idx == 10:
    break

Toxic sentence: he had steel balls too !
Detoxified sentence: he was brave too!
Gold label: he was brave too!

Toxic sentence: dude should have been taken to api , he would be right at home with all the other knuckleheads there
Detoxified sentence: dude should have been taken to api, he would be right at home with all the other people there
Gold label: It would have been good if he went to api. He would fit in.

Toxic sentence: im not gonna sell the fucking picture , i just want to contribute to the fucking article .
Detoxified sentence: I'm not gonna sell the picture, I just want to contribute to the article.
Gold label: I'm not gonna sell the picture, i just want to contribute to the article.

Toxic sentence: the garbage that is being created by cnn and other news agencies is outrageous .
Detoxified sentence: the news that is being created by cnn and other news agencies is outrageous.
Gold label: the news that is being created by cnn and other news agencies is outrageous.

Toxic sent

In [ ]:
batch_size = 32
lr = 1e-5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_epochs = 1

In [ ]:
for epoch in range(num_epochs):
  model.train()
  total_loss = 0.0
  for i,batch in enumerate(dataloader):
    input_ids, detoxified_ids, input_mask = batch

    model_outputs = model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)
    loss = model_outputs.loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    # Print average loss for the epoch
    print(f"Train Loss: {loss.item():.4f}")
  average_loss = total_loss / len(dataloader)
  print(f"Epoch {epoch+1}, Average Loss: {average_loss:.4f}")


Train Loss: 10.2882
Train Loss: 9.6240
Train Loss: 9.2656
Train Loss: 8.5633
Train Loss: 8.1424
Train Loss: 7.6346
Train Loss: 7.0775
Train Loss: 7.0625
Train Loss: 6.5217
Train Loss: 6.1345
Train Loss: 5.7747
Train Loss: 5.6319
Train Loss: 5.1128
Train Loss: 4.8510
Train Loss: 4.7847
Train Loss: 3.9672
Train Loss: 4.1133
Train Loss: 3.6300
Train Loss: 3.4004
Train Loss: 3.1923
Train Loss: 3.1422
Train Loss: 2.8551
Train Loss: 2.6599
Train Loss: 2.3238
Train Loss: 2.6036
Train Loss: 2.5442
Train Loss: 2.4412
Train Loss: 1.8643
Train Loss: 2.1754
Train Loss: 2.3395
Train Loss: 2.8254
Train Loss: 2.2949
Epoch 1, Average Loss: 4.8388
